In [1]:
import pandas as pd
from datasets import load_dataset
from torchtuning import koalpaca_dataset, kowiki_dataset, custom_chat_dataset
from torchtune.models.llama3 import llama3_tokenizer
import numpy as np, json

In [2]:
tokenizer = llama3_tokenizer('checkpoints/Meta-Llama-3-8B-Instruct/original/tokenizer.model')

In [3]:
with open('a.json', 'r') as f:
    a = json.load(f)

In [4]:
aa = []
for x in a:
    aa.append({'user': x['eng'][0]['content'], 'assistant': x['eng'][1]['content']})
    aa.append({'user': x['kor'][0]['content'], 'assistant': x['kor'][1]['content']})
    aa.append({'user': x['eng'][0]['content'], 'assistant': x['kor'][1]['content']})
    aa.append({'user': x['kor'][0]['content'], 'assistant': x['eng'][1]['content']})

In [5]:
pd.DataFrame(aa).to_csv('a.csv')

In [22]:
ds = custom_chat_dataset(
    tokenizer=tokenizer,
    source='csv',
    max_seq_len=1536,
    split='train',
    data_files='a.csv'
)

In [23]:
ds._data[0]

{'Unnamed: 0': 0,
 'user': 'What is the best way to learn Korean?',
 'assistant': 'There\'s no single "best" way to learn Korean, as it depends on your learning style, goals, and available resources. However, here\'s a comprehensive approach that combines different methods:\n\n1. **Start with the Basics:**\n    - Learn the Korean alphabet (Hangul) first. It\'s relatively simple and phonetic, making it easier to start reading and writing.  You can find many resources online for learning Hangul, including apps, YouTube videos, and websites.\n    - Familiarize yourself with basic Korean grammar. This includes sentence structure, verb conjugations, and common particles.\n\n2. **Immerse Yourself:**\n    - **Language Exchange:** Find a Korean speaker to practice with. This could be online through language exchange platforms or in person through meetups. Speaking with a native speaker is invaluable for pronunciation and fluency.\n    - **Korean Media:**  Watch Korean dramas, movies, and varie

In [24]:
k = ds[0]

In [25]:
ds._convert_to_messages(ds._data[4])

[Message(role='user', content='Can you write a poem about a lonely lighthouse keeper?', masked=True, ipython=False, eot=True),
 Message(role='assistant', content="The wind howls a mournful tune,\nAcross the gray and churning sea,\nThe lonely lighthouse keeper's moon,\nHangs heavy in the endless spree.\n\nHe watches as the waves crash down,\nWith silent, watchful, steady gaze,\nNo solace in the bustling town,\nJust endless sky and misty haze.\n\nThe beacon's light, a guiding hand,\nTo ships that sail through stormy night,\nA lonely man, a lonely land,\nHe keeps the darkness burning bright.\n\nHe dreams of warmth, of laughter's chime,\nOf faces kind and friendly hand,\nHe dreams of home, of love's sweet rhyme,\nA life beyond this lonely sand.", masked=False, ipython=False, eot=True)]

In [26]:
kds = kowiki_dataset(
    tokenizer=tokenizer,
    max_seq_len=1536,
    data_files='data/20240520/kowikitext_20240520.csv',
    split='train'
)

In [28]:
kds._data[0]

{'text': '= 플라톤 =\n\n플라톤(, 플라톤, "넓은, 어깨 폭이 넓은";Diogenes Laertius 3.4; ; Seneca, Epistulae, VI, 58, 30: illi nomen latitudo pectoris fecerat., ; 기원전 428년/기원전 427년 또는 기원전 424년/기원전 424년  ~ 기원전 348년/기원전 347년)은 다양한 서양 학문에 영향력 있는 그리스의 철학자이자 사상가, 객관적 관념론(objective idealism)의 창시자이다. 그는 소크라테스의 제자, 아리스토텔레스의 스승이며, 대학의 원형인 고등 교육 기관 ‘아카데메이아’의 교육자이다.\n\n플라톤은 아카데메이아에서 폭넓은 주제를 강의하였으며, 특히 정치학, 윤리학, 형이상학, 인식론 등 많은 철학적 논점에 관해 저술하였다. 플라톤 저술 가운데 가장 중요한 게 그의 대화편이다. 비록 일부 편지는 단지 그의 이름을 붙여서 전해지지만, 플라톤이 쓴 진짜 대화편은 모두 온전하게 전해진 것으로 여긴다. 그러나 학자들의 합의에 따라, 그리스인들이 플라톤의 것으로 생각하는 ‘알키비아데스 I’과 ‘클레이토폰’ 등과 같은 대화편은 의심스러우며 ‘데모도코스’와 ‘알키비아데스 II’ 등과 같은 대화편은 대개 위조되었다고 판단한다. 편지는 거의 모두 위조되었다고 대개 여기며, 예외로 일곱 번째 편지만 위조되지 않았을 가능성이 있다.\n\n소크라테스는 플라톤의 대화편에 자주 등장하는 주요 등장인물이다. 플라톤의 대화편에 있는 내용과 주장 가운데 무엇이 소크라테스의 것이고 플라톤의 것인지에 관해 많은 논쟁이 있었다. 왜냐하면 소크라테스는 남긴 저술이 없기 때문이다. 이 문제를 종종 ‘소크라테스의 문제’라고 부른다. 그러나 플라톤은 소크라테스의 가르침에 많은 영향을 받았다는 게 확실하다. 따라서 수많은 플라톤의 아이디어, 적어도 그의 초기 연구는 소크라테스에게서 가져오거나 발전시켰을 것이다.\n\n그가 이성 우위의 전통을 가진 서양 철학에 미친 영향은 더할 수 없이 크다.